In [60]:
import pymysql
import jieba
import jieba.analyse
from imp import reload
import matplotlib.pyplot as plt
import csv
import gensim
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
from gensim import models,corpora
from gensim.models import doc2vec, ldamodel
from gensim.models import CoherenceModel
import random
import warnings
from keybert import KeyBERT
from math import log, exp
import sys

In [61]:
def ReadTweetsByTime(dataname,date_list):
    doclist=[]
    if dataname=='GPT': #搜集的推特关于GPT的数据集，时间从一月四号持续到三月29号
       with open('../推特GPT/GPT.csv',encoding="utf-8") as f:
         f_csv = csv.reader(f)
         pos = 1
         for row in f_csv:
            if  len(row)>=6:  #控制聚类文本数目并处理缺失数据,全部跑太多了跑不了
                #print(row[0][:10])  #取到日期
                if row[0][:10] in date_list: #判断时间
                    #print(row[0])
                    doclist.append(row[2])
                    pos+=1
         print(f'总数量{pos}')
    #return random.sample(doclist,200)
    global data_number
    if data_number!=0:
        return random.sample(doclist,data_number)
    else:  #表示数据全部要
        data_number=len(doclist)
        return doclist

In [62]:
def GetKeyBertWord(dataname,date_list):
    #使用keybert方法，获取关键词
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    doclist=ReadTweetsByTime(dataname,date_list)
    dic={}
    list_word=[]
    list_weight=[]
    pos=0
    for item in doclist:
        #print(item)
        pos=pos+1
        global data_number
        #print(pos/data_number)
        print("\r", end="")
        print("取词中: {}".format(pos/data_number),end="") #进度条
        #有点慢，两千条数据要跑十分钟
        keywords_phrase = kw_model.extract_keywords(item, 
                                     keyphrase_ngram_range=(1,1),   #这个就是范围,可以考虑词组,本实验暂时只考虑词
                                     stop_words='english', 
                                     highlight=False, 
                                     top_n=3) 
        keywords_phrase_list= list(keywords_phrase)#结果，返回的是列表包含元组的形式[(),()]
        #print(keywords_phrase_list)
        
        for word in keywords_phrase_list:
            #print(word[0])
            #把数据加入字典,要处理1.去除全局的词  2.判断是否已经存在于字典中
            if word[0] not in ['chatgpt','gpt','#chatgpt','chat']:
              if word[0] not in list_word:
                list_word.append(word[0])
                list_weight.append(word[1])
              else:
                index=list_word.index(word[0])
                list_weight[index]=list_weight[index]+word[1]
        #print(keywords_list)
        
    #最终返回两个数据，字典形式的结果和原始数据
    dic['word']=list_word
    dic['weight']=list_weight
    print(' ')#这个空行主要是为了和进度条配合完成换行,后面也有
    return dic,doclist

In [63]:
def GetBrim(list_word):
    #获取节点间的边信息，主要使用相似度
    list_link_dic=[]
    global threshold_similarity  #全局变量，边的门限
    model = models.word2vec.Word2Vec.load('word2vec.model')  #使用相似度试试
    for i in range(0,len(list_word)):
        dic={}
        for j in range(i+1,len(list_word)):
            try:
               similarity = model.wv.similarity(list_word[i],list_word[j])
            except:
               similarity=0   #主要是不存在某个词的情况
            if(similarity>=threshold_similarity):  #判断点的权重
                dic={}
                dic['Source']=i
                dic['Target']=j
                dic['Type']=0
                dic['Weight']=similarity
                list_link_dic.append(dic)
        #print(f"\r算边中:{(i+1)/len(list_word)}",end=' ')
        print("\r", end="")
        print(f'算边进度为{(i+1)/len(list_word)}',end="",flush=True)
    print('\n完成算边')
    return list_link_dic

In [64]:
def WriteCSV_gephi(file_path,time_name,list_word_dic,list_link_dic):#写两个csv，在gephi中完成画图
    #具体格式，第一个节点csv，依次为'id','label','type','typelabel','size'
    #          第二个连线csv 依次为'源','目标','Label','Id','Weight'
    global write_version
    file_node=file_path+write_version+time_name+'-node.csv'
    file_link=file_path+write_version+time_name+'-link.csv'
    list_node=[]
    list_link=[]
    #print('开始写入文件')
    for item in list_word_dic:
        node=[]
        node.append(item['pos'])
        node.append(item['word'])
        node.append('1') #暂时不分type
        node.append('1') #暂时不分type
        node.append(item['weight'])
        list_node.append(node)
    for item in list_link_dic:
        link=[]
        link.append(item['Source'])
        link.append(item['Target'])
        link.append(item['Type'])
        link.append(item['Weight'])
        list_link.append(link)
    #在写入边的时候进行判断，舍弃低权边
    #list_link.sort(key=lambda ele: ele[3], reverse=True)#降序
    #list_link=list_link[0:300]  #取前多少条
    with open(file_node, 'w', encoding='UTF8',newline="") as f_node:
        writer = csv.writer(f_node) 
        writer.writerow(['id','label','type','typelabel','size'])
        for item in list_node:
            writer.writerow(item)
    with open(file_link, 'w', encoding='UTF8',newline="") as f_link:
        writer = csv.writer(f_link) 
        writer.writerow(['Source','Target','Type','Weight'])
        for item in list_link:
            writer.writerow(item)

In [65]:
def GetNode(sorted_word,sorted_weight):
    #工具函数，将词和权重打包
    list_word_dic=[]
    for i in range(0,len(sorted_word)):
        dic={}
        dic['pos']=i
        dic['word']=sorted_word[i]
        dic['weight']=sorted_weight[i]
        list_word_dic.append(dic)
    return list_word_dic

In [66]:
threshold_similarity=0.8   #成边的相似度 门限值
data_number=0  #这个值是考虑的文本数量，如果为0则是全部文本。否则只取data_number条。  这个参数也帮助进度条
write_version='/成图csvV3/'  #控制最终结果写入那个文件夹
random.seed(100)
source_data='GPT' #数据源
filepath_name='E:\code\舆情idea2\推特GPT'#文件夹根目录位置
model = models.word2vec.Word2Vec.load('word2vec.model')
split_number=50   #截取前多少个词
#需要操作的日期



<>:6: DeprecationWarning: invalid escape sequence '\c'
<>:6: DeprecationWarning: invalid escape sequence '\c'
C:\Users\12193\AppData\Local\Temp\ipykernel_14228\371125812.py:6: DeprecationWarning: invalid escape sequence '\c'
  filepath_name='E:\code\舆情idea2\推特GPT'#文件夹根目录位置


In [67]:
ls_date=[
    #{'time':'1月6日起','date':['2023-01-06','2023-01-07','2023-01-08'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'1月9日起','date':['2023-01-09','2023-01-10','2023-01-11'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'1月12日起','date':['2023-01-12','2023-01-13','2023-01-14'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'1月15日起','date':['2023-01-15','2023-01-16','2023-01-17'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'1月18日起','date':['2023-01-18','2023-01-19','2023-01-20'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'1月21日起','date':['2023-01-21','2023-01-22','2023-01-23'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'1月24日起','date':['2023-01-24','2023-01-25','2023-01-26'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'1月27日起','date':['2023-01-27','2023-01-28','2023-01-29'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'1月30日起','date':['2023-01-30','2023-01-31','2023-02-01'],'tips':'全数据，边为相似度,门限0.8'},
    #{'time':'2月2日起','date':['2023-02-02','2023-02-03','2023-02-04'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'2月5日起','date':['2023-02-05','2023-02-06','2023-02-07'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'2月8日起','date':['2023-02-08','2023-02-09','2023-02-10'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'2月11日起','date':['2023-02-11','2023-02-12','2023-02-13'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'2月14日起','date':['2023-02-14','2023-02-15','2023-02-16'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'2月17日起','date':['2023-02-17','2023-02-18','2023-02-19'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'2月20日起','date':['2023-02-20','2023-02-21','2023-02-22'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'2月23日起','date':['2023-02-23','2023-02-24','2023-02-25'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'2月26日起','date':['2023-02-26','2023-02-27','2023-02-28'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月1日起','date':['2023-03-01','2023-03-02','2023-03-03'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月4日起','date':['2023-03-04','2023-03-05','2023-03-06'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月7日起','date':['2023-03-07','2023-03-08','2023-03-09'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月10日起','date':['2023-03-10','2023-03-11','2023-03-12'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月13日起','date':['2023-03-13','2023-03-14','2023-03-15'],'tips':'全数据，边为相似度,门限0 .8'},
    {'time':'3月16日起','date':['2023-03-16','2023-03-17','2023-03-18'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月19日起','date':['2023-03-19','2023-03-20','2023-03-21'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月22日起','date':['2023-03-22','2023-03-23','2023-03-24'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月25日起','date':['2023-03-25','2023-03-26','2023-03-27'],'tips':'全数据，边为相似度,门限0.8'},
    {'time':'3月28日起','date':['2023-03-28','2023-03-29'],'tips':'全数据，边为相似度,门限0.8'},
]

In [68]:
for item in ls_date:
    #不知道为什么，print \r实现的进度条会超过一点
    print(f"开始执行:{item['time']}")
    data_number=0  #这个值是考虑的文本数量，如果为0则是全部文本。否则只取data_number条。  这个参数也帮助进度条
    dic,doclist = GetKeyBertWord(source_data,item['date'])
    sorted_word = sorted(dic['word'], key=lambda x: dic['weight'][dic['word'].index(x)],reverse=True)  #根据weight操作word排序
    sorted_weight=sorted(dic['weight'],reverse=True)
    list_link_dic=GetBrim(sorted_word[:split_number])  #数据越多越慢，不建议超过两千条  #只取权重在前面的词
    list_word_dic=GetNode(sorted_word[:split_number],sorted_weight[:split_number]) #拼接出词
    WriteCSV_gephi(filepath_name,item['time']+str(data_number)+item['tips'],list_word_dic,list_link_dic)

开始执行:2月5日起
总数量22807
取词中: 1.09995615188985354444 
算边进度为1.08
完成算边
开始执行:2月8日起
总数量24369
取词中: 1.09995896257386745666 
算边进度为1.08
完成算边
开始执行:2月11日起
总数量17815
取词中: 1.0 994386437633326456
算边进度为1.08
完成算边
开始执行:2月14日起
总数量18674
取词中: 1.09994644674128423654 
算边进度为1.08
完成算边
开始执行:2月17日起
总数量16116
取词中: 1.09993794601303136466 
算边进度为1.08
完成算边
开始执行:2月20日起
总数量17955
取词中: 1.09994430210538043456 
算边进度为1.08
完成算边
开始执行:2月23日起
总数量15077
取词中: 1.09993366940833123563 
算边进度为1.08
完成算边
开始执行:2月26日起
总数量13957
取词中: 1.09992834623101183653 
算边进度为1.08
完成算边
开始执行:3月1日起
总数量17548
取词中: 1.09994301020117497566 
算边进度为1.08
完成算边
开始执行:3月4日起
总数量13556
取词中: 1.0 992622648469296555
算边进度为1.08
完成算边
开始执行:3月7日起
总数量15354
取词中: 1.09993486614993815546 
算边进度为1.08
完成算边
开始执行:3月10日起
总数量12913
取词中: 1.09992255266418844665 
算边进度为1.08
完成算边
开始执行:3月13日起
总数量25127
取词中: 1.09996020058903127457 
算边进度为1.08
完成算边
开始执行:3月16日起
总数量25264
取词中: 1.0 996041641926923653
算边进度为1.08
完成算边
开始执行:3月19日起
总数量20432
取词中: 1.09995105476971274564 
算边进度为1.08
完成算边
开始执行:3月22日起
总数量24490
取词中: 1.09995